In [5]:
import os
os.environ['SIRIUS_URL_CONSTS'] = 'http://127.0.0.1'

In [6]:
%gui qt5
%matplotlib qt5

from functools import partial as _partial
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
import matplotlib.gridspec as mpl_gs
import matplotlib.cm as cmap
import matplotlib.colors as mpl_colors
from matplotlib import rcParams
rcParams.update(
    {'font.size': 16,'lines.linewidth': 2, 'axes.grid': True})

import pyaccel
import mathphys
from pymodels import si
from apsuite.dynap import DynapXY
from apsuite.optics_analysis import TuneCorr

In [15]:
model = si.create_accelerator()
model.cavity_on = True
model.radiation_on = False
dynap = DynapXY(model)

In [18]:
tunecorr = TuneCorr(model, 'SI')

In [19]:
tunecorr.correct_parameters([49.186188, 14.2300])

1

In [20]:
twi, _ = pyaccel.optics.calc_twiss(dynap._acc)
twi.mux[-1]/2/np.pi, twi.muy[-1]/2/np.pi

(49.18574477838651, 14.230530180928666)

In [16]:
dynap.params.turn_by_turn = True
dynap.params.x_nrpts = 70
dynap.params.y_nrpts = 30
dynap.params.x_min = -0.012
dynap.params.x_max = 0.00
dynap.params.y_min = 0
dynap.params.y_max = 0.004
print(dynap)

nrturns      : 512
turn_by_turn : True
x_nrpts      : 5
y_nrpts      : 5
x_min [m]    : -0.012
x_max [m]    : 0
y_min [m]    : 0
y_max [m]    : 0.004
de_offset    : 0
intnux       : 49.00 (for graphs)
intnuy       : 14.00 (for graphs)



In [17]:
dynap.do_tracking()

In [18]:
dynap.process_data()

In [24]:
# dynap.save_data('/home/fernando/delta_tune2.pickle')
dynap.load_and_apply('/home/fernando/delta_tune2.pickle')
dynap.process_data()

In [22]:
dynap.load_and_apply('/home/fernando/nominal.pickle')
dynap.process_data()

# Diffusion Plot

In [19]:
bounds = [
    dynap.params.intnux, dynap.params.intnux + 0.5,
    dynap.params.intnuy, dynap.params.intnuy + 0.5]
orders = [6, ]
symmetry = 1
resons = dynap.calc_resonances_for_bounds(bounds, orders=orders, symmetry=symmetry)

In [20]:
map2xy = dynap.map_resons_to_xyplane(resons, maxdist=1e-3)
numb = [x.size for x, _ in map2xy]
for i, (res, nu) in enumerate(zip(resons, numb)):
    print(f'{i:2}: {res}  {nu:3}')
filter_resons = [reson for reson, n in zip(resons, numb) if n > 10]

 0: (-2, 4, -43)    0
 1: (-1, 1, -35)    0
 2: (-3, 3, -106)    0
 3: (-4, 2, -167)    0
 4: (1, 5, 121)    0
 5: (-3, 0, -148)    0
 6: (-2, 4, -41)    0
 7: (5, 1, 261)    0
 8: (-5, 1, -231)    0
 9: (1, 5, 120)    0
10: (-2, 0, -99)    0
11: (5, 1, 260)    0
12: (-5, 1, -232)    0
13: (3, 0, 148)    0
14: (1, 1, 64)    0
15: (1, 5, 122)    0
16: (3, 3, 191)    0
17: (2, 0, 99)    0
18: (5, 1, 262)    0
19: (3, 3, 190)    0
20: (-2, 1, -85)    0
21: (-5, 1, -233)    0
22: (-1, 2, -20)    0
23: (-4, 2, -169)    0
24: (1, 5, 119)    0
25: (5, 1, 259)    0
26: (-1, 2, -21)    2
27: (-6, 0, -295)    0
28: (-3, 3, -104)    0
29: (0, 6, 85)    1
30: (4, 2, 225)    0
31: (-1, 0, -49)    0
32: (-2, 1, -84)    0
33: (6, 0, 295)    0
34: (1, 1, 63)    0
35: (4, 2, 227)    0
36: (1, 0, 49)    0
37: (2, 4, 155)    1
38: (-1, 5, 22)    0
39: (1, 2, 77)    0
40: (-1, 5, 23)    0
41: (0, 3, 43)    0
42: (0, 1, 14)    0
43: (1, 2, 78)    2
44: (0, 2, 29)    0
45: (2, 4, 157)    0
46: (2, 1, 112)  

In [21]:
f, ax, ay = dynap.make_figure_diffusion(contour=True, resons=resons, maxdist=1e-3)
# f, ax, ay = dynap.make_figure_diffusion(contour=True, resons=filter_resons, maxdist=1e-3)
f.show()
# f = dynap.make_figure_diffusion(contour=False)

# X and Y map to tune plot

In [29]:
f = dynap.make_figure_xandy_map_in_tune_plot(resons)